In [1]:
import sys
import sagemaker
#!{sys.executable} -m pip install "sagemaker==1.72.0" smdebug

In [2]:
import boto3

account_id = boto3.client("sts").get_caller_identity().get("Account")
ecr_repository = "keynol-ml-docker-image"
tag = ":latest"

region = boto3.session.Session().region_name

uri_suffix = "amazonaws.com"
if region in ["cn-north-1", "cn-northwest-1"]:
    uri_suffix = "amazonaws.com.cn"
byoc_image_uri = "{}.dkr.ecr.{}.{}/{}".format(account_id, region, uri_suffix, ecr_repository + tag)
ecr_repo_base = "{}.dkr.ecr.{}.{}".format(account_id, region, uri_suffix)

In [4]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin $ecr_repo_base

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [5]:
!docker build -t $ecr_repository docker

Sending build context to Docker daemon  3.584kB
Step 1/7 : FROM ubuntu:18.04
18.04: Pulling from library/ubuntu

55322776: Pull complete .71MB/26.71MBBDigest: sha256:0fedbd5bd9fb72089c7bbca476949e10593cebed9b1fb9edf5b79dbbacddd7d6
Status: Downloaded newer image for ubuntu:18.04
 ---> 5a214d77f5d7
Step 2/7 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          python          nginx          default-jre          libenchant1c2a          libmysqlclient-dev          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Running in 8cbad25af0b5
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [26.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic InRelease [242 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1433 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2372 kB]
Get:6 

In [6]:
!docker images

REPOSITORY                           TAG                 IMAGE ID            CREATED             SIZE
edo-edm-sagemaker-scikit-learn-dev   latest              6ae3c6da20b6        1 second ago        3.79GB
ubuntu                               18.04               5a214d77f5d7        2 weeks ago         63.1MB


In [7]:
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
!docker tag {ecr_repository + tag} $byoc_image_uri
!docker push $byoc_image_uri

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
The push refers to repository [308904643607.dkr.ecr.us-east-1.amazonaws.com/edo-edm-sagemaker-scikit-learn-dev]

eaa957d7: Preparing 
5698d5ae: Preparing 
5760e5ff: Preparing 
3bb89faf: Preparing 
604951b8: Preparing 
eaa957d7: Pushed   2.955GB/2.932GBPushing  55.76MB/443.1MBPushing  1.065GB/2.932GBPushing  1.649GB/2.932GBPushing  1.671GB/2.932GBPushing   1.74GB/2.932GBPushing  1.933GB/2.932GBPushing  2.434GB/2.932GBPushing   2.64GB/2.932GBlatest: digest: sha256:0e2aaf772f31c9e980e4937b806570319c77b555b453014c75d114ac77f8287c size: 1585


In [10]:
!docker images

REPOSITORY                           TAG                 IMAGE ID            CREATED             SIZE
edo-edm-sagemaker-scikit-learn-dev   latest              6ae3c6da20b6        9 minutes ago       3.79GB
ubuntu                               18.04               5a214d77f5d7        2 weeks ago         63.1MB
